<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Ames Housing Data and Kaggle Challenge

### Data Import

In [1]:
#Imports
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',20 )
pd.set_option('display.max_columns', 300)
import scipy.stats

In [2]:
#read in training data
train_clean = pd.read_csv("../datasets/clean_data/train_clean.csv")

## Feature Engineering & Preprocessing<a id='feature_engineering'></a>

In [3]:
#create copy for editing
train = train_clean.copy()

In [5]:
#encode ordinal variables with custom values
train = train.replace({"lot_shape" : {"IR3" : 1, "IR2" : 1, "IR1" : 2, "Reg" : 3},
                      "utilities" : {"ELO" : 1, "NoSeWa" : 1, "NoSewr" : 2, "AllPub" : 3},
                      "land_slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 2},
                      "exter_qual" : {"Po" : 1, "Fa" : 1, "TA": 2, "Gd": 3, "Ex" : 4},
                      "exter_cond" : {"Po" : 1, "Fa" : 1, "TA": 2, "Gd": 3, "Ex" : 4},
                      "bsmt_qual" : {"None" : 0, "Po" : 1, "Fa" : 1, "TA": 1, "Gd" : 2, "Ex" : 3},
                      "bsmt_cond" : {"None" : 0, "Po" : 1, "Fa" : 1, "TA" : 1, "Gd" : 2, "Ex" : 3},
                      "bsmt_exposure" : {"None" : 0, "No" : 1, "Mn" : 1, "Av": 2, "Gd" : 3},
                      "bsmtfin_type_1" : {"None" : 0, "Unf" : 1, "LwQ": 1, "Rec" : 2, "BLQ" : 3, 
                                         "ALQ" : 3, "GLQ" : 4},
                       "bsmtfin_type_2" : {"None" : 0, "Unf" : 1, "LwQ": 1, "Rec" : 2, "BLQ" : 3, 
                                         "ALQ" : 3, "GLQ" : 4},
                       "heating_qc" : {"Po" : 1, "Fa" : 1, "TA" : 2, "Gd" : 2, "Ex" : 3},
                       "electrical" : {"Mix" : 0, "FuseP" : 1, "FuseF": 1, "FuseA" : 2, "SBrkr" : 2},
                       "kitchen_qual" : {"Po" : 1, "Fa" : 1, "TA" : 2, "Gd" : 2, "Ex" : 3},
                       "functional" : {"Sal" : 1, "Sev" : 1, "Maj2" : 2, "Maj1" : 2, "Mod": 3, 
                                       "Min2" : 3, "Min1" : 4, "Typ" : 4},
                       "garage_finish" : {"None" : 0, "Unf" : 1, "RFn" : 1, "Fin" : 2},
                       "garage_qual" : {"None" : 0, "Po" : 1, "Fa" : 1, "TA" : 2, "Gd" : 2, "Ex" : 3},
                        "garage_cond" : {"None" : 0, "Po" : 1, "Fa" : 1, "TA" : 2, "Gd" : 2, "Ex" : 3},
                       "paved_drive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "pool_qc" : {"None" : 0, "Fa" : 1, "TA" : 1, "Gd" : 2, "Ex" : 3},
                       "fence" : {"None" : 0, "MnWw" : 1, "GdWo" : 1, "MnPrv" : 2, "GdPrv" : 3},
                       })

In [6]:
#add new features for modeling
#some found through trial and error with testing using unregularized linear regression

train['total_sq_ft'] = train["gr_liv_area"] + train["total_bsmt_sf"]
train["total_bathroom"] = train["full_bath"] + train["bsmt_full_bath"] \
                                + 0.5 * (train["half_bath"] + train["bsmt_half_bath"])

train["ga_oq_sf"] = train["garage_area"] * train["overall_qual"] * train["total_sq_ft"]
train["lf_sf"] = train["lot_frontage"] * train["total_sq_ft"]  
train["op_sf3"] = train["open_porch_sf"] * train["total_sq_ft"] *train["total_sq_ft"] * train["total_sq_ft"] 
train["op_oq3"] = train["open_porch_sf"] * train["overall_qual"] * train["overall_qual"] * train["overall_qual"] 
train["oc_sf"] = train["overall_cond"] * train["total_sq_ft"] 
train["kq_f_lf"] = train["kitchen_qual"] * train["fireplaces"] * train["lot_frontage"]
train["pd_oq"] = train["paved_drive"] * train["overall_qual"] 
train["gq_sf"] = train["garage_qual"] * train["total_sq_ft"] 


In [7]:
#Check skew of variables for potential log transformation candidates
skew_values = pd.DataFrame(train.select_dtypes(np.number).columns, columns = ["variable"])
skew_values["absolute_skew"] = skew_values["variable"].map(lambda x: abs(scipy.stats.skew(train[x])))
skew_transform = list(skew_values.loc[skew_values['absolute_skew'] > .5,"variable"])

In [8]:
train.select_dtypes(include=[np.number]).describe().T

,count,mean,std,min,25%,50%,75%,max
id,2049.0,1473.676428,844.248064,1.0,753.0,1481.0,2199.0,2930.0
lot_frontage,2049.0,69.757247,21.786571,21.0,60.0,70.0,80.0,313.0
lot_area,2049.0,10024.677892,6608.511749,1300.0,7500.0,9405.0,11500.0,159000.0
lot_shape,2049.0,2.601269,0.548986,1.0,2.0,3.0,3.0,3.0
utilities,2049.0,2.998536,0.049389,1.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...
op_oq3,2049.0,16189.354807,28945.012860,0.0,0.0,5250.0,21266.0,502000.0
oc_sf,2049.0,14043.857784,4611.383773,1002.0,11340.0,13620.0,16385.0,44667.0
kq_f_lf,2049.0,95.619717,120.178723,0.0,0.0,68.0,160.0,800.0
pd_oq,2049.0,11.446559,4.257599,0.0,10.0,12.0,14.0,20.0


In [9]:
#apply log transformation with log1p due to min values being 0 for some variables
for col in skew_transform:
    train[col] = np.log1p(train[col])

In [10]:
#Keep only variables to be used for modeling

train.drop(columns = ["id", "ms_zoning", "lot_area", "street", 
                     "condition_2", "house_style", "overall_cond", "year_built",
                     "year_remod/add", "roof_style", "roof_matl", "exterior_1st", "exterior_2nd", 
                     "mas_vnr_type", "exter_cond", "bsmt_qual", "bsmt_cond",
                     "bsmtfin_type_1", "bsmtfin_sf_1", "bsmtfin_sf_1", "bsmtfin_sf_2", "bsmtfin_type_2", "bsmt_unf_sf",
                     "total_bsmt_sf", "heating", "electrical", "1st_flr_sf", 
                     "2nd_flr_sf", "low_qual_fin_sf", "gr_liv_area", "bsmt_full_bath", "bsmt_half_bath", 
                     "full_bath", "half_bath", "bedroom_abvgr", "kitchen_abvgr", "totrms_abvgrd",
                     "fireplace_qu", "garage_type", "garage_yr_blt", "garage_area", "garage_cond",
                      "enclosed_porch", "3ssn_porch",
                     "screen_porch", "pool_area", "pool_qc", "fence", "misc_feature", "misc_val",
                     "yr_sold", "sale_type"], inplace = True)

In [11]:
#one hot encode nominal variables to be used in model
train = pd.get_dummies(data = train, columns = ["alley", "lot_config", "condition_1", "bldg_type",
                                               "foundation", "central_air", "mo_sold", "ms_subclass",
                                               "land_contour", "neighborhood"], drop_first = True)

In [13]:
#save edited data set
train.to_csv("../datasets/clean_data/train_feature_ohe.csv", index = False)

### Next Step:<a id='next_step'></a>

#### [Model Tuning](./03_Model_Tuning.ipynb)